# Bedrock Model Testing (Optional)

**This notebook is NOT part of the workshop.** It's a diagnostic tool for testing which Bedrock models are accessible in your AWS account.

Use this to:
- Verify IAM permissions for Bedrock APIs
- Test which Claude models are enabled in your account
- Troubleshoot model access issues before running the main labs

In [ ]:
import boto3

# Show current identity
sts = boto3.client("sts")
identity = sts.get_caller_identity()
print(f"Account: {identity['Account']}")
print(f"ARN:     {identity['Arn']}")

In [ ]:
# Test Bedrock API access (no model invocation)
bedrock = boto3.client("bedrock", region_name="us-west-2")

# List foundation models - tests bedrock:ListFoundationModels permission
models = bedrock.list_foundation_models(byProvider="Anthropic")
claude_models = [m["modelId"] for m in models["modelSummaries"]]

print(f"Found {len(claude_models)} Anthropic models:")
for m in claude_models[:5]:
    print(f"  - {m}")
if len(claude_models) > 5:
    print(f"  ... and {len(claude_models) - 5} more")

print("\n✓ Bedrock API access working!")

In [ ]:
# Test model invocation (requires Marketplace subscription)
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-west-2")

try:
    response = bedrock_runtime.converse(
        modelId="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        messages=[{"role": "user", "content": [{"text": "Say OK"}]}],
        inferenceConfig={"maxTokens": 10}
    )
    print(f"Response: {response['output']['message']['content'][0]['text']}")
    print("\n✓ Model invocation working!")
except Exception as e:
    print(f"✗ Model invocation failed: {type(e).__name__}")
    print(f"  {e}")
    print("\n  This usually means Marketplace permissions are needed.")
    print("  Add to your role: aws-marketplace:ViewSubscriptions, aws-marketplace:Subscribe")

## Comprehensive Permission Test

Tests all Bedrock permissions required for Lab 4 and reports results.

In [ ]:
"""Comprehensive Bedrock Permission Test for Lab 4."""
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import os

# Load config
load_dotenv("../CONFIG.txt")
REGION = os.getenv("REGION", "us-west-2")
MODEL_ID = os.getenv("MODEL_ID", "us.anthropic.claude-3-sonnet-20240229-v1:0")

def test_permission(name, test_func):
    """Test a single permission and return result."""
    try:
        test_func()
        print(f"  [PASS] {name}")
        return True
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_msg = e.response['Error']['Message']
        print(f"  [FAIL] {name}")
        print(f"         Error: {error_code}")
        print(f"         {error_msg[:100]}...")
        return False
    except Exception as e:
        print(f"  [FAIL] {name}")
        print(f"         Error: {type(e).__name__}: {str(e)[:100]}")
        return False

print(f"Testing permissions for Lab 4")
print(f"Model:  {MODEL_ID}")
print(f"Region: {REGION}")
print("=" * 60)

bedrock = boto3.client("bedrock", region_name=REGION)
bedrock_runtime = boto3.client("bedrock-runtime", region_name=REGION)

results = {}

# Bedrock Control Plane
print("\n--- Bedrock Control Plane ---")
results["bedrock:ListFoundationModels"] = test_permission(
    "bedrock:ListFoundationModels",
    lambda: bedrock.list_foundation_models(byProvider="Anthropic")
)

results["bedrock:GetFoundationModel"] = test_permission(
    "bedrock:GetFoundationModel",
    lambda: bedrock.get_foundation_model(modelIdentifier="anthropic.claude-3-sonnet-20240229-v1:0")
)

# Bedrock Runtime
print("\n--- Bedrock Runtime ---")
results["bedrock-runtime:Converse"] = test_permission(
    "bedrock-runtime:Converse",
    lambda: bedrock_runtime.converse(
        modelId=MODEL_ID,
        messages=[{"role": "user", "content": [{"text": "Hi"}]}],
        inferenceConfig={"maxTokens": 5}
    )
)

results["bedrock-runtime:InvokeModel"] = test_permission(
    "bedrock-runtime:InvokeModel",
    lambda: bedrock_runtime.invoke_model(
        modelId=MODEL_ID,
        body='{"anthropic_version":"bedrock-2023-05-31","max_tokens":5,"messages":[{"role":"user","content":[{"type":"text","text":"Hi"}]}]}'
    )
)

# Summary
print("\n" + "=" * 60)
passed = sum(results.values())
total = len(results)
print(f"SUMMARY: {passed}/{total} permissions verified")
print("=" * 60)

if passed == total:
    print("\n[OK] All required permissions for Lab 4 are configured!")
else:
    failed = [k for k, v in results.items() if not v]
    print(f"\n[WARN] Missing permissions: {', '.join(failed)}")
    print("\nSee IAM_PERMS.md for instructions on adding these permissions.")

## Verified Working Models

Test the 4 verified working Claude models.

In [ ]:
import boto3

def test_model(model_id: str, region: str = "us-west-2"):
    """Test a specific Bedrock model with both Converse and InvokeModel APIs.
    
    Args:
        model_id: The Bedrock model ID to test
        region: AWS region (default: us-west-2)
    """
    print(f"Testing model: {model_id}")
    print(f"Region: {region}")
    print("=" * 60)
    
    bedrock_runtime = boto3.client("bedrock-runtime", region_name=region)
    
    # Test Converse API
    print("\n--- Converse API ---")
    try:
        response = bedrock_runtime.converse(
            modelId=model_id,
            messages=[{"role": "user", "content": [{"text": "Say OK"}]}],
            inferenceConfig={"maxTokens": 10}
        )
        result = response['output']['message']['content'][0]['text']
        print(f"  [PASS] Response: {result}")
    except Exception as e:
        print(f"  [FAIL] {type(e).__name__}: {str(e)[:100]}")
    
    # Test InvokeModel API
    print("\n--- InvokeModel API ---")
    try:
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body='{"anthropic_version":"bedrock-2023-05-31","max_tokens":10,"messages":[{"role":"user","content":[{"type":"text","text":"Say OK"}]}]}'
        )
        import json
        result = json.loads(response['body'].read())['content'][0]['text']
        print(f"  [PASS] Response: {result}")
    except Exception as e:
        print(f"  [FAIL] {type(e).__name__}: {str(e)[:100]}")
    
    print("\n" + "=" * 60)

print("test_model() function defined")

In [ ]:
# Test Claude 3 Sonnet (base)
test_model("anthropic.claude-3-sonnet-20240229-v1:0")

In [ ]:
# Test Claude 3 Sonnet (US cross-region)
test_model("us.anthropic.claude-3-sonnet-20240229-v1:0")

In [ ]:
# Test Claude Sonnet 4.5 (US cross-region)
test_model("us.anthropic.claude-sonnet-4-5-20250929-v1:0")

In [ ]:
# Test Claude Sonnet 4.5 (Global cross-region)
test_model("global.anthropic.claude-sonnet-4-5-20250929-v1:0")